In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import json

with open('intents.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses
0,loneliness,[I'm experiencing a deep sense of emptiness an...,[The Lonely City: Adventures in the Art of Bei...
1,Overwhelm,[I'm feeling overwhelmed and just need some sp...,[The Power of Now: A Guide to Spiritual Enligh...
2,General sadness,"[I feel down, I feel sad, I'm finding it hard ...",[The Noonday Demon: An Atlas of Depression by ...
3,Emotional distress,"[I'm feeling really down today., I'm feeling s...","[The Body Keeps the Score: Brain, Mind, and Bo..."
4,Work stress,[I'm feeling burnt out and exhausted from work...,"[The 5 AM Club: Own Your Morning, Elevate Your..."
5,Academic stress,"[I'm so stressed out about my exams., I have s...",[How to Read a Book by Mortimer Adler and Char...
6,General stress,"[I'm feeling like I'm at my breaking point., I...",[Why Zebras Don't Get Ulcers: An Updated Guide...
7,worthless,"[I feel so worthless., No one likes me., I can...",[The Gifts of Imperfection: Let Go of Who You ...
8,depressed,"[I can't take it anymore, I am so depressed, I...",[Feeling Good: The New Mood Therapy by David D...
9,anxious,"[I feel so anxious., I'm so anxious because of...",[Feeling Good: The New Mood Therapy by David D...


In [3]:
dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

df = pd.DataFrame.from_dict(dic)
df

,tag,patterns,responses
0,loneliness,I'm experiencing a deep sense of emptiness and...,[The Lonely City: Adventures in the Art of Bei...
1,loneliness,It's difficult to cope with the feeling of bei...,[The Lonely City: Adventures in the Art of Bei...
2,loneliness,I'm struggling to find meaning in my daily lif...,[The Lonely City: Adventures in the Art of Bei...
3,loneliness,The feeling of isolation is overwhelming and m...,[The Lonely City: Adventures in the Art of Bei...
4,loneliness,I'm starting to question whether I'm worthy of...,[The Lonely City: Adventures in the Art of Bei...
...,...,...,...
2222,suicide,I'm feeling like there's no way to change my c...,"[Reasons to Stay Alive by Matt Haig, The Happi..."
2223,suicide,"I'm experiencing a lot of anger or resentment,...","[Reasons to Stay Alive by Matt Haig, The Happi..."
2224,suicide,I'm feeling like I'm a burden on society or th...,"[Reasons to Stay Alive by Matt Haig, The Happi..."
2225,suicide,I'm struggling with feelings of nostalgia or l...,"[Reasons to Stay Alive by Matt Haig, The Happi..."


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

X shape =  (2227, 42)
y shape =  (2227,)
num of classes =  14


In [6]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(Input(shape=(X.shape[1])))
model.add(Embedding(input_dim=vacab_size+1, output_dim=100, mask_zero=True))
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(LayerNormalization())
model.add(LSTM(32))
model.add(LayerNormalization())
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(LayerNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation="softmax"))
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

model.summary()
plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 42, 100)           177300    
                                                                 
 lstm (LSTM)                 (None, 42, 32)            17024     
                                                                 
 layer_normalization (LayerN  (None, 42, 32)           64        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 42, 32)            8320      
                                                                 
 layer_normalization_1 (Laye  (None, 42, 32)           64        
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 32)                8

In [7]:
model_history = model.fit(x=X,
                          y=y,
                          batch_size=50,
                          callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)],
                          epochs=30)

Epoch 1/30
45/45 [==============================] - 19s 83ms/step - loss: 2.4659 - accuracy: 0.2595
Epoch 2/30
45/45 [==============================] - 5s 119ms/step - loss: 1.5151 - accuracy: 0.5168
Epoch 3/30
45/45 [==============================] - 10s 224ms/step - loss: 0.9926 - accuracy: 0.6870
Epoch 4/30
45/45 [==============================] - 5s 102ms/step - loss: 0.7389 - accuracy: 0.7746
Epoch 5/30
45/45 [==============================] - 4s 98ms/step - loss: 0.6030 - accuracy: 0.8145
Epoch 6/30
45/45 [==============================] - 5s 101ms/step - loss: 0.4629 - accuracy: 0.8550
Epoch 7/30
45/45 [==============================] - 4s 99ms/step - loss: 0.3661 - accuracy: 0.8801
Epoch 8/30
45/45 [==============================] - 5s 106ms/step - loss: 0.3528 - accuracy: 0.8828
Epoch 9/30
45/45 [==============================] - 5s 105ms/step - loss: 0.2935 - accuracy: 0.9084
Epoch 10/30
45/45 [==============================] - 5s 102ms/step - loss: 0.2337 - accuracy: 0.9237


In [8]:
import re
import random

def model_responce(query): 
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', query)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]
    tag = df[df['tag'] == tag]['tag'].values
    print("you: {}".format(query))
    print("model: {}".format(random.choice(tag)))


In [9]:
model_responce("I am feeling stressed");

1/1 [==============================] - 5s 5s/step
you: I am feeling stressed
model: Overwhelm


In [1]:
import pickle

In [8]:
file_name = 'solutionChallenge.pkv'
pickle.dump(model,open(file_name,'wb'))

NameError: name 'model' is not defined

In [12]:
load_model = pickle.load(open('solutionChalange.pkv','rb'))

In [13]:
model_responce("I am feeling like I want to kill my Teacher");

1/1 [==============================] - 0s 37ms/step
you: I am feeling like I want to kill my Teacher
model: suicide
